In [95]:
from datetime import timedelta
from datetime import datetime
import pandas as pd
from datetime import timezone
import numpy as np

In [96]:
import json

config = json.load(open("../configure.json"))
start_date = datetime.fromisoformat(config['start_time']).astimezone(tz=timezone.utc).replace(tzinfo=None).timestamp()
end_date = datetime.fromisoformat(config['end_time']).astimezone(tz=timezone.utc).replace(tzinfo=None).timestamp()

In [97]:
apple_news = pd.read_pickle("pkls/apple_news_2gram.pkl", compression='infer')
amazon_news = pd.read_pickle("pkls/amazon_news_2gram.pkl", compression='infer')
apple_chart = pd.read_pickle("pkls/apple_stock_chart.pkl", compression='infer')
amazon_chart = pd.read_pickle("pkls/apple_stock_chart.pkl", compression='infer')

In [98]:
apple_news

,published_time,article_content,word_tokens,features
12886,1.514876e+09,here are some things going on today in your wo...,"[go, today, world, tech, bellweth, tech, march...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
13571,1.514876e+09,shutterstock photo\nstocks indexes opened the ...,"[photo, stock, new, year, first, session, chee...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
11837,1.514876e+09,by ryan vlastelica\nto simply match the market...,"[simpli, match, market, may, need, take, lot, ...","[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
13804,1.514876e+09,what happened shares of many optical networkin...,"[mani, optic, decemb, accord, data, p, global,...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
15737,1.514876e+09,by nigam arora\nthe practical way to take adva...,"[practic, way, take, advantag, januari, effect...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...
63653,1.549568e+09,introduction welcome to my monthly update for ...,"[introduct, welcom, month, updat, dividend, gr...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
63392,1.549572e+09,amazon is running away with the smart-speaker ...,"[amazon, run, away, market, chris, motley, foo...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
62662,1.549574e+09,"the "" fast money "" traders shared their first ...","[fast, money, first, market, open, pete, buyer...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
62647,1.549580e+09,investors lately have taken a somewhat jaundic...,"[late, taken, somewhat, view, toward, mani, te...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, ..."


In [99]:
amazon_news

,published_time,article_content,word_tokens,features
12886,1.514876e+09,here are some things going on today in your wo...,"[go, today, world, tech, bellweth, tech, march...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
13571,1.514876e+09,shutterstock photo\nstocks indexes opened the ...,"[photo, stock, new, year, first, session, chee...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
11837,1.514876e+09,by ryan vlastelica\nto simply match the market...,"[simpli, match, market, may, need, take, lot, ...","[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
13804,1.514876e+09,what happened shares of many optical networkin...,"[mani, optic, decemb, accord, data, p, global,...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
15737,1.514876e+09,by nigam arora\nthe practical way to take adva...,"[practic, way, take, advantag, januari, effect...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...
63653,1.549568e+09,introduction welcome to my monthly update for ...,"[introduct, welcom, month, updat, dividend, gr...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
63392,1.549572e+09,amazon is running away with the smart-speaker ...,"[amazon, run, away, market, chris, motley, foo...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
62662,1.549574e+09,"the "" fast money "" traders shared their first ...","[fast, money, first, market, open, pete, buyer...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
62647,1.549580e+09,investors lately have taken a somewhat jaundic...,"[late, taken, somewhat, view, toward, mani, te...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, ..."


In [100]:
amazon_news['trend'] = [0 for i in range(0, len(amazon_news))]
amazon_news

,published_time,article_content,word_tokens,features,trend
12886,1.514876e+09,here are some things going on today in your wo...,"[go, today, world, tech, bellweth, tech, march...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
13571,1.514876e+09,shutterstock photo\nstocks indexes opened the ...,"[photo, stock, new, year, first, session, chee...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
11837,1.514876e+09,by ryan vlastelica\nto simply match the market...,"[simpli, match, market, may, need, take, lot, ...","[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
13804,1.514876e+09,what happened shares of many optical networkin...,"[mani, optic, decemb, accord, data, p, global,...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
15737,1.514876e+09,by nigam arora\nthe practical way to take adva...,"[practic, way, take, advantag, januari, effect...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
...,...,...,...,...,...
63653,1.549568e+09,introduction welcome to my monthly update for ...,"[introduct, welcom, month, updat, dividend, gr...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
63392,1.549572e+09,amazon is running away with the smart-speaker ...,"[amazon, run, away, market, chris, motley, foo...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
62662,1.549574e+09,"the "" fast money "" traders shared their first ...","[fast, money, first, market, open, pete, buyer...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
62647,1.549580e+09,investors lately have taken a somewhat jaundic...,"[late, taken, somewhat, view, toward, mani, te...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, ...",0


In [101]:
import time, datetime

index = 0
row = []
minutes = 1440

form = "%d-%m-%Y %H:%M"
trends = []


def get_time_to_trend(chart, time_interval):
    time_map = {}
    if minutes == time_interval:
        for chart_index, chart_row in chart.iterrows():
            day = datetime.datetime.utcfromtimestamp(chart_row['timestamp'])
            day = (day - datetime.timedelta(hours=day.hour, minutes=day.minute, seconds=day.second,
                                            microseconds=day.microsecond))
            timestamp = day.strftime(form)
            time_map[timestamp] = chart_row['trends']
    return time_map


time_to_trend = get_time_to_trend(amazon_chart, 1440)

try:
    for index, row in amazon_news.iterrows():
        day = datetime.datetime.utcfromtimestamp(row['published_time'])
        day = (day - datetime.timedelta(hours=day.hour, minutes=day.minute, seconds=day.second,
                                        microseconds=day.microsecond))
        timestamp = day.strftime(form)
        while timestamp not in time_to_trend:
            day += datetime.timedelta(hours=24)
            timestamp = day.strftime(form)
        trends.append(int(time_to_trend[timestamp]))
except Exception as e:
    print(e)
    print("it fails")
    print(row, index)
amazon_news['trend'] = trends

date value out of range
it fails
published_time                                          1549168860.0
article_content    / investing why huawei matters, fomc decision,...
word_tokens        [decis, watch, market, recon, think, agre, inc...
features           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
trend                                                              0
Name: 63868, dtype: object 63868


ValueError: Length of values (6079) does not match length of index (6148)

In [ ]:
amazon_news

In [ ]:
import numpy

amazon_news.drop("word_tokens", axis=1, inplace=True)
amazon_news.drop("article_content", axis=1, inplace=True)
amazon_news.drop("published_time", axis=1, inplace=True)

amazon_news

In [ ]:
import pickle

pwd = "/Users/k2rth1k/swm/stock_prediction_g24_p3/"
with open(pwd + 'data_preprocess/pkls/amazon_data_for_training.pkl', 'wb') as f:
    pickle.dump(amazon_news, f)